In [12]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr
import torchbnn as bnn

In [13]:
#import creep data
creep_df = pd.read_csv('Ti_alloys_dataset.csv')
creep_df

,Ti,Al,V,Fe,C,N,H,O,Sn,Nb,...,Cr,Solution treated temp(cel),ST time (h),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc),creep_rupture_life
0,87.8750,6.75,4.50,0.40,0.100,0.050,0.1250,0.20,0.00,0.0,...,0.0,690,4.0,1050,0.5,700,319.0,2.090000e-03,15.80,0.01000
1,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,319.0,3.240000e-06,0.00,0.01167
2,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,250.0,4.400000e-05,0.00,0.03050
3,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,500,520.0,4.430000e-05,0.00,0.04620
4,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,700,319.0,2.130000e-03,0.00,0.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,85.9870,6.20,0.00,0.00,0.021,0.004,0.0080,0.03,1.95,0.0,...,0.0,900,1.0,580,8.0,520,310.0,1.200000e-04,0.00,1160.00000
173,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,940,4.0,455,379.0,9.160000e-07,0.00,1619.00000
174,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,1030,0.5,455,379.0,1.270000e-06,0.00,1744.00000
175,89.4100,6.00,4.00,0.25,0.080,0.050,0.0100,0.20,0.00,0.0,...,0.0,0,0.0,0,0.0,538,103.0,9.000000e-07,22.30,4681.00000


In [14]:
creep_df.head()

,Ti,Al,V,Fe,C,N,H,O,Sn,Nb,...,Cr,Solution treated temp(cel),ST time (h),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc),creep_rupture_life
0,87.875,6.75,4.50,0.40,0.100,0.050,0.125,0.20,0.0,0.0,...,0.0,690,4.0,1050,0.5,700,319.0,0.002090,15.8,0.01000
1,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,600,319.0,0.000003,0.0,0.01167
2,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,600,250.0,0.000044,0.0,0.03050
3,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,500,520.0,0.000044,0.0,0.04620
4,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,700,319.0,0.002130,0.0,0.05000


In [15]:
rm_state = 123
test_size = 0.6

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:12]), np.array(creep_df.iloc[:,12]), shuffle=True, test_size=test_size, random_state=rm_state)

idx = np.arange(len(y))

In [16]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [17]:
train_ratio = 0.1

X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [18]:
X = X.astype(dtype=np.float32)
X_test =X_test.astype(np.float32)
X_train =X_train.astype(np.float32)

y = y.astype(dtype=np.float32)
y_test = y_test.astype(dtype=np.float32)
y_train = y_train.astype(dtype=np.float32)

X = torch.from_numpy(X)
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)
y = torch.from_numpy(y)

y_train = torch.unsqueeze(y_train, dim=1)
y_test = torch.unsqueeze(y_test, dim=1)
y = torch.unsqueeze(y, dim=1)

In [19]:
n_iter = 6
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []
pcc_random = []
r2_random = []
num_training_data = []

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

#build the model 
model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=12, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=100),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.06, in_features=100, out_features=1),
)

mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01
optimizer = optim.SGD(model.parameters(), lr=0.001, nesterov=True, momentum=0.95)

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        print()
   
        q_points_var = np.argpartition(y_pred_unc_pool_var, -10)[-10:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]

    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")
    
    num_training_data.append(len(idx_train_var))

    for step in range(3000):
        y_pred_var = model(X_train_var)
        
        mse = mse_loss(y_pred_var, y_train_var)
        kl = kl_loss(model)
        cost = mse + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if step % 1000==0:
        print('MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()))

    n_samples = 1000

    # compute predictions
    y_preds_var = [model(X_test).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_var = np.array(y_preds_var)
    # mean and standard deviation
    y_pred_test_var = np.mean(y_preds_var, axis=0)
    y_pred_unc_test_var = np.std(y_preds_var, axis=0)
    # y_test = y_test.detach().numpy()


    # get the idx pool var
    y_preds_pool_var = [model(X[idx_pool_var]).clone().detach().numpy() for _ in range(n_samples)]
    y_preds_pool_var = np.array(y_preds_pool_var)
    y_pred_unc_pool_var = np.squeeze(np.std(y_preds_pool_var, axis=0))

    print('PCC_test_score', pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    print('R2_test_score', r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    print('RMSE', np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    print('MAE', np.mean(abs(np.squeeze(y_test.numpy()) - np.squeeze(y_pred_test_var))))

    pcc_variance.append(pearsonr(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))[0])
    r2_variance.append(r2_score(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var)))
    rmse_variance.append(np.sqrt(mean_squared_error(np.squeeze(y_test.numpy()), np.squeeze(y_pred_test_var))))
    mae_variance.append(np.mean(abs(np.squeeze(y_test.numpy())- np.squeeze(y_pred_test_var))))

Performing iteration : 0
Number of training data with variance: 7
Number of pooling data with variance: 63
PCC_test_score 0.21766453337552855
R2_test_score -0.04320873352986965
RMSE 0.12606795
MAE 0.096095674
Performing iteration : 1

Number of training data with variance: 17
Number of pooling data with variance: 53
PCC_test_score 0.2248223170748247
R2_test_score -0.14880800614437195
RMSE 0.13229482
MAE 0.09078091
Performing iteration : 2

Number of training data with variance: 27
Number of pooling data with variance: 43
PCC_test_score 0.7642124161206849
R2_test_score 0.5547542244211207
RMSE 0.082360506
MAE 0.058506552
Performing iteration : 3

Number of training data with variance: 37
Number of pooling data with variance: 33
PCC_test_score 0.8745478107213789
R2_test_score 0.7395642931968522
RMSE 0.062989675
MAE 0.044107195
Performing iteration : 4

Number of training data with variance: 47
Number of pooling data with variance: 23
PCC_test_score 0.9102041859363733
R2_test_score 0.81539

'\n    # random sampling\n    if i != 0:\n        # select 10 random data points\n        q_points_ran = np.random.choice(np.arange(len(idx_pool_ran)), size=10)\n        # indices of those points in idx_pool\n        idx_pool_train_ran = idx_pool_ran[q_points_ran]\n\n        idx_train_ran = np.append(idx_train_ran, idx_pool_train_ran)\n        idx_pool_ran = np.delete(idx_pool_ran, q_points_ran)\n        X_train_ran = X[idx_train_ran]\n        y_train_ran = y[idx_train_ran]\n\n    print(f"Number of training data with random: {len(idx_train_ran)}")\n    print(f"Number of pooling data with random: {len(idx_pool_ran)}")\n\n    for step in range(3000):\n\n\n        y_pred_ran = model(X_train_ran)\n        mse = mse_loss(y_pred_ran, y_train_ran)\n        kl = kl_loss(model)\n        cost = mse + kl_weight*kl\n\n        optimizer.zero_grad()\n        cost.backward()\n        optimizer.step()\n\n    if step % 1000==0:\n        print(\'MSE : %2.2f, KL : %2.2f\' % (mse.item(), kl.item()))\n\n  

In [20]:
# plt.plot(num_iteration, r2_variance, 'b')
# plt.plot(num_iteration, r2_random, 'r')
# plt.show()
import pickle

In [23]:
with open('AL_BNN_VarInf_TI.pkl', 'wb') as f:
    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

In [24]:
pkl_file = open('AL_BNN_VarInf_TI.pkl', 'rb')  
test = pickle.load(pkl_file)